In [ ]:
import torch
from insightface.app import FaceAnalysis
from insightface.model_zoo.arcface_onnx import ArcFaceONNX
from insightface.utils import face_align
import cv2
import numpy as np
import sys
import sys
import os 
sys.path.append(os.path.abspath(os.path.join("", "..")))
sys.path.append("/home/ubuntu/AutoLoRADiscovery/discover_lora_diffusion/weights2weights/")
import torch
import warnings
warnings.filterwarnings("ignore")
from PIL import Image
from discover_lora_diffusion.weights2weights.lora_w2w import LoRAw2w
from discover_lora_diffusion.weights2weights.utils import unflatten
from diffusers import DiffusionPipeline 
from peft import PeftModel
from peft.utils.save_and_load import load_peft_weights
# 

# import clip
import wandb
# import clip
# device = "cuda" if torch.cuda.is_available() else "cpu"
# clip_model, preprocess = clip.load("ViT-L/14", device="cuda")

from transformers import AutoImageProcessor, AutoModel
preprocess = AutoImageProcessor.from_pretrained('facebook/dinov2-large')
dino_model = AutoModel.from_pretrained('facebook/dinov2-large')
dino_model = dino_model.cuda()


def rtn_face_get(self, img, face):
    aimg = face_align.norm_crop(img, landmark=face.kps, image_size=self.input_size[0])
    #print(cv2.imwrite("aimg.png", aimg))
    face.embedding = self.get_feat(aimg).flatten()
    face.crop_face = aimg
    return face.embedding

ArcFaceONNX.get = rtn_face_get
app = FaceAnalysis(name="buffalo_l", providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(256, 256))

def crop_face(img, face_ratio=1.0):
    cv2_img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    faces = app.get(cv2_img)
    try:
        bbox = faces[0]['bbox']

        # get largest possible square around center point
        w,h = img.width, img.height
        box_w = bbox[2] - bbox[0]
        box_h = bbox[3] - bbox[1]
        if box_w < box_h:
            diff = box_h - box_w
            bbox[0] -= diff // 2
            bbox[2] += diff // 2
        else:
            diff = box_w - box_h
            bbox[1] -= diff // 2
            bbox[3] += diff // 2
        dist_to_left = bbox[0]
        dist_to_right = w - bbox[2]
        dist_to_top = bbox[1]
        dist_to_bottom = h - bbox[3]
        min_dist = min(dist_to_left, dist_to_right, dist_to_top, dist_to_bottom)
        min_dist = int(min_dist * face_ratio) - 1
        bbox = [bbox[0]-min_dist, bbox[1]-min_dist, bbox[2]+min_dist, bbox[3]+min_dist]
        face_image = img.crop(bbox)

        return face_image, faces[0].embedding
    except:
        return None

In [ ]:
import torch
idens = torch.load("/mnt/rd/identity_df.pt", map_location="cpu")

In [ ]:
idens.iloc[5]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(emb)
plt.show()


plt.plot(_emb_norm)
plt.show()


In [ ]:
_emb_norm = emb / np.linalg.norm(emb)

In [ ]:
# show me face of /mnt/rd/img_align_celeba/000005.jpg
import requests
img1 = Image.open("/home/ubuntu/AutoLoRADiscovery/junk/1691527680903.jpeg")
img2 = Image.open(requests.get("https://foivospar-arc2face.hf.space/file=/tmp/gradio/f9e31998958141179c8c265d3b91ba8892014605/image.png", stream=True).raw)
face1, emb1 = crop_face(img1)
face2, emb2 = crop_face(img2)

In [ ]:
face1.show(), face2.show()

In [ ]:
# cosine similarity
def cos_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

cos_sim(emb1, emb2)

In [ ]:
# show me face of /mnt/rd/img_align_celeba/000005.jpg
img = Image.open("/mnt/rd/img_align_celeba/000005.jpg")
face, emb = crop_face(img)
face.show(), img.show()

In [ ]:
import sys
sys.path.append("/home/ubuntu/AutoLoRADiscovery/discover_lora_diffusion/weights2weights/")

In [ ]:
import torch
weights = torch.load("/mnt/rd/all_weights.pt", map_location="cpu")#.bfloat16().cuda()#.half()

In [ ]:
V = torch.load("/mnt/rd/V.pt", map_location="cuda")#.half()
mean = torch.load("/mnt/rd/mean.pt", map_location="cuda")#.half()
std = torch.load("/mnt/rd/std.pt", map_location="cuda")#.half()

In [ ]:
with torch.no_grad():
    x = weights - mean.bfloat16()
    x = x / std.bfloat16()
    actual_pcas = torch.matmul(x, V.bfloat16())

In [ ]:
weights.max(), weights.min(), weights.mean(), weights.std()

In [ ]:
from discover_lora_vae.models import LoraVAE, config
import torch

lora_vae = LoraVAE(
    input_dim=99_648,
    latent_dim=4096,
).cuda()

lora_vae.load_state_dict(torch.load("/mnt/rd/model-out-2/checkpoint-48000", map_location="cuda"))

In [ ]:
import tqdm
test_weight = weights#.float()
batch_size = 4096
with torch.no_grad():
    test_pca = torch.zeros(test_weight.shape[0], 4096, device="cuda")
    # reconstructed = torch.zeros(test_weight.shape[0], 99_648, device="cpu")
    for i in tqdm.tqdm(range(0, test_weight.shape[0], batch_size)):
        merged = lora_vae.encode( 
            lora_vae.apply_std_on_weights(test_weight[i:i+batch_size].float().cuda()),
            # mean_logvar=True#
              )[0]#.chunk(2, dim=-1)[0]
        # decoded_lora = lora_vae.deapply_std_on_weights(lora_vae.decode(merged))

        # merged = torch.cat([mean, logvar], dim=-1)
        test_pca[i:i+batch_size] = merged
        # reconstructed[i:i+batch_size] = decoded_lora
    # test_pca = ((test_weight - mean) / (std)) @ V

In [ ]:
import matplotlib.pyplot as plt

plt.plot(lora_vae.apply_std_on_weights(weights[0:1]).cpu().numpy()[0])  

In [ ]:
weights[0].min(), weights[0].max(), weights[0].mean(), weights[0].std()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(weights[0].cpu().numpy()/0.0152)  

In [ ]:
import matplotlib.pyplot as plt

plt.plot(weights[0].cpu().numpy())  

# PCA and rest

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Assuming your latent vectors are in a numpy array called 'latent_vectors'
# with shape (100, 4096)

# 1. Perform PCA
pca = PCA()
pca_result = pca.fit_transform(test_pca[:10000, :].cpu().numpy())

# 2. Calculate cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# 3. Plot cumulative explained variance
plt.figure(figsize=(10, 6))
plt.plot(cumulative_variance_ratio)
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('PCA: Cumulative Explained Variance Ratio')
plt.grid(True)
plt.show()

# 4. Plot explained variance ratio for first 50 components
plt.figure(figsize=(10, 6))
plt.bar(range(50), pca.explained_variance_ratio_[:50])
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('PCA: Explained Variance Ratio of First 50 Components')
plt.show()

# 5. Scatter plot of first two principal components
plt.figure(figsize=(10, 8))
plt.scatter(pca_result[:, 0], pca_result[:, 1])
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('PCA: First Two Principal Components')
plt.show()

# 6. Print the number of components needed to explain 95% of the variance
n_components_95 = np.argmax(cumulative_variance_ratio >= 0.95) + 1
print(f"Number of components needed to explain 95% of variance: {n_components_95}")

In [ ]:
import umap
import numpy as np
import matplotlib.pyplot as plt

# Assuming your data is in a numpy array called 'latent_vectors'
# with shape (10000, 4096)

# Create a UMAP model
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)

# Fit and transform the data
embedding = reducer.fit_transform(test_pca[:1000, :].cpu().numpy())

# Plot the result
plt.figure(figsize=(12, 10))
plt.scatter(embedding[:, 0], embedding[:, 1])
plt.title('UMAP projection of the latent space')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.show()

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Assuming your data is in a numpy array called 'latent_vectors'
tsne = TSNE(n_components=2, random_state=42)
latent_2d = tsne.fit_transform(test_pca[:1000, :].cpu().numpy())

plt.figure(figsize=(10, 8))
plt.scatter(latent_2d[:, 0], latent_2d[:, 1])
plt.title('t-SNE visualization of latent space')
plt.show()

In [ ]:
# numpy_weights.shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Assuming your latent vectors are in a numpy array called 'latent_vectors'
# with shape (100, 4096)

# 1. Perform PCA
pca = PCA()
pca_result = pca.fit_transform(test_pca[:, :].cpu().numpy())

In [ ]:
pca_result.shape

# Diffusion generate

In [ ]:
import torch
from discover_lora_diffusion.models import LoraDiffusion
import diffusers

config = {
        "_class_name": "UnCLIPScheduler",
        "_diffusers_version": "0.17.0.dev0",
        "clip_sample": True,
        "clip_sample_range": 10.0,  # Adjusted to match your data range
        "num_train_timesteps": 1000,  # Reduced from 1000
        "prediction_type": "sample",
        "variance_type": "fixed_small_log"
}

# scheduler = diffusers.UnCLIPScheduler.from_config("kandinsky-community/kandinsky-2-2-prior", subfolder="scheduler")
scheduler = diffusers.UnCLIPScheduler.from_config(config)

# scheduler.set_timesteps(50)
lora_diffusion = LoraDiffusion(
        data_dim=99_648,
        model_dim=768,
        ff_mult=3,
        chunks=1,
        act=torch.nn.SiLU,
        num_blocks=6,
        layers_per_block=4,
    ).cuda()
lora_diffusion.load_state_dict(torch.load("/mnt/rd/diff_lora_mdl/checkpoint-clpface_newdataset22_small_50step_denoise_reluflat-134000", map_location="cuda"))

In [ ]:
import os
from PIL import Image
from IPython.display import display, Image as IPImage
import io
import copy
from discover_lora_diffusion.weights2weights.lora_w2w import LoRAw2w
from discover_lora_diffusion.weights2weights.utils import unflatten
from diffusers import DiffusionPipeline , logging
from peft import PeftModel
from peft.utils.save_and_load import load_peft_weights
import torch
weight_dimensions = torch.load("/mnt/rd/weight_dimensions.pt")
device = "cuda:0"

pipe = DiffusionPipeline.from_pretrained("stablediffusionapi/realistic-vision-v51", 
                                            torch_dtype=torch.float16,safety_checker = None,
                                            requires_safety_checker = False).to(device)
pipe_unet = pipe.unet                               
def generate_from_weights(weight):
    if os.path.exists("/mnt/rd/inference_lora_2"):
        os.system("rm -rf /mnt/rd/inference_lora_2")

    unflatten(weight.unsqueeze(0).detach().clone(), weight_dimensions, "/mnt/rd/inference_lora_2")
    logging.disable_progress_bar()
    

    pipe.set_progress_bar_config(disable=True)

    pipe.unet = copy.deepcopy(pipe_unet)
    pipe.unet = PeftModel.from_pretrained(pipe.unet, "/mnt/rd/inference_lora_2/unet", adapter_name="identity1")
    adapters_weights1 = load_peft_weights("/mnt/rd/inference_lora_2/unet", device="cuda:0")
    pipe.unet.load_state_dict(adapters_weights1, strict = False)
    pipe.unet.to("cuda", torch.float16)

    # torch rng 
    generator = torch.Generator(device="cuda")
    generator.manual_seed(0)
    negative_prompt = "low quality, blurry, unfinished, cartoon"
    images = pipe(["sks person with dog", "sks person with dog"],
                   negative_prompt=[negative_prompt]*2,
    
     height=640, width=640, num_inference_steps=50, guidance_scale=2.5, generator=generator).images
    canvas = Image.new('RGB', (640, 645*2))
    for i, image in enumerate(images):
        canvas.paste(image, (0, 650*i))

    # return images[0]
    return canvas   
    # save as low res image
    imgbyte = io.BytesIO()
    canvas.save(imgbyte, format="JPEG", quality=20)
    imgbyte.seek(0)



    display(IPImage(data=imgbyte.read(), width=640*4))

In [ ]:
_weights = weights[0:1]

In [ ]:
_new_weight = lora_vae.deapply_std_on_weights(lora_vae.decode(lora_vae.encode(lora_vae.apply_std_on_weights(_weights.cuda()))[0]))

In [ ]:
_new_weight.shape
show_the_lats(_weights)

In [ ]:
noisy = torch.randn(1, 99_648).cuda()
# _latents = scheduler.add_noise(lora_vae.apply_std_on_weights(weights[0:1]).cuda(), noisy, torch.Tensor([20]).type(torch.int).cuda())#torch.randn(1, 99_648).cuda()  * scheduler.init_noise_sigma 
latents = noisy
# Create an unconditioned embedding for CFG
# uncond_face_emb = torch.zeros_like(face_emb1)
def show_the_lats(latents, return_img=False):
    canvas = generate_from_weights(latents[0].detach().clone())
    if return_img:
        return canvas

    import io
    from IPython.display import display, Image as IPImage
    imgbyte = io.BytesIO()
    canvas.save(imgbyte, format="JPEG", quality=20)
    imgbyte.seek(0)


    display(IPImage(data=imgbyte.read(), width=canvas.width))
# prev_latents = [latents]
# latents, _ = lora_diffusion(face_emb1)
mid_render = []
with torch.no_grad():
    for t in scheduler.timesteps:
        # latents = scheduler.scale_model_input(latents, t)
        noise_pred, pred_cond = lora_diffusion(
            latents, 
            t=t.unsqueeze(0).cuda().half(), 
            face_embeddings=torch.Tensor(idens.iloc[1].values[:-1]).unsqueeze(0).cuda()
        )
        
        latents = scheduler.step(noise_pred, t, latents, return_dict=False)[0]
        if ((t % 3 == 0) and (t < 30)) or ((t == (len(scheduler.timesteps) - 1))):
            print(t)
            img = show_the_lats(lora_vae.deapply_std_on_weights(latents), return_img=True)
            mid_render.append(img)
latents = lora_vae.deapply_std_on_weights(latents)



show_the_lats(latents)

canvas = Image.new('RGB', (mid_render[0].width*len(mid_render), mid_render[0].height))
for i, image in enumerate(mid_render):
    canvas.paste(image, (mid_render[0].width*i, 0))

import io
from IPython.display import display, Image as IPImage
imgbyte = io.BytesIO()
canvas.save(imgbyte, format="JPEG", quality=20)
imgbyte.seek(0)


display(IPImage(data=imgbyte.read(), width=canvas.width))

# show_the_lats(weights[0:1])



In [ ]:
idens.iloc[1]

# Rest

In [ ]:
import os
from PIL import Image
from IPython.display import display, Image as IPImage
import io
import copy
from discover_lora_diffusion.weights2weights.lora_w2w import LoRAw2w
from discover_lora_diffusion.weights2weights.utils import unflatten
from diffusers import DiffusionPipeline , logging
from peft import PeftModel

from peft.utils.save_and_load import load_peft_weights
import torch
weight_dimensions = torch.load("/mnt/rd/weight_dimensions.pt")
device = "cuda:0"

pipe = DiffusionPipeline.from_pretrained("stablediffusionapi/realistic-vision-v51", 
                                            torch_dtype=torch.float16,safety_checker = None,
                                            requires_safety_checker = False).to(device)
pipe_unet = pipe.unet                               
def generate_from_weights(weight):
    if os.path.exists("/mnt/rd/inference_lora"):
        os.system("rm -rf /mnt/rd/inference_lora")

    unflatten(weight.unsqueeze(0).detach().clone(), weight_dimensions, "/mnt/rd/inference_lora")
    logging.disable_progress_bar()
    

    pipe.set_progress_bar_config(disable=True)

    pipe.unet = copy.deepcopy(pipe_unet)
    pipe.unet = PeftModel.from_pretrained(pipe.unet, "/mnt/rd/inference_lora/unet", adapter_name="identity1")
    adapters_weights1 = load_peft_weights("/mnt/rd/inference_lora/unet", device="cuda:0")
    pipe.unet.load_state_dict(adapters_weights1, strict = False)
    pipe.unet.to("cuda", torch.float16)

    # torch rng 
    generator = torch.Generator(device="cuda")
    # generator.manual_seed(0)
    negative_prompt = "low quality, blurry, unfinished, cartoon"
    images = pipe(["sks person"]*2,
                  negative_prompt=[negative_prompt]*2,
    
     height=640, width=640, num_inference_steps=50, guidance_scale=2.5, generator=generator).images
    canvas = Image.new('RGB', (640*2, 640*2))
    for i, image in enumerate(images):
        canvas.paste(image, (640*i, 320))

    # return canvas

    # save as low res image
    imgbyte = io.BytesIO()
    canvas.save(imgbyte, format="JPEG", quality=20)
    imgbyte.seek(0)

    display(IPImage(data=imgbyte.read(), width=640*4))

In [ ]:
def generate_from_latent(latent):
    # convert latent to weights
    weights = lora_vae.deapply_std_on_weights(lora_vae.decode(latent.float().unsqueeze(0).cuda())).squeeze(0)
    return generate_from_weights(torch.tensor(weights, device="cuda:0"))

In [ ]:
random_latent = torch.randn(4096).cuda()
generate_from_latent(random_latent)

In [ ]:
import numpy as np
import torch

# Assuming test_pca is your tensor of latent vectors
with torch.no_grad():
    latent_np = torch.Tensor(test_pca)[:, :].float().cuda()#.cpu().numpy()

    # Compute the covariance matrix
    cov_matrix = torch.cov(latent_np.T)

    # Perform eigendecomposition
    eigenvalues, eigenvectors = torch.linalg.eig(cov_matrix)
    eigenvalues = eigenvalues.cpu().numpy()
    eigenvectors = eigenvectors.cpu().numpy()

    # Sort eigenvectors by eigenvalues in descending order
    idx = eigenvalues.argsort()[::-1]   
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

def manipulate_latent(latent, eigenvector, magnitude):
    return latent + torch.tensor(eigenvector * magnitude, device=latent.device)



# Display interpolation images

In [ ]:
def generate_from_pca(pc):
    inverted = pca.inverse_transform(pc.cpu().numpy())
    inverted = torch.tensor(inverted, device="cuda")
    return generate_from_latent(inverted)
    # with torch.no_grad():
        
    #     weights = pc.bfloat16() @ V.T.bfloat16()
    #     weights = weights * mean.bfloat16()
    #     weights = weights + mean.bfloat16()
    #     generate_from_weights(torch.tensor(weights, device="cuda:0"))

In [ ]:
eigenvectors[:,0].min(), eigenvectors[:,0].max()

In [ ]:
# # Generate images
from tqdm import trange
base_latent = test_pca[0]
# original_image = generate_from_latent(base_latent)
# manipulated_image = generate_from_latent(manipulated_latent)


# Interpolation along an eigenvector
def interpolate_eigenvector(base_latent, eigenvector, start=-10, end=10, steps=3):
    for magnitude in np.linspace(start, end, steps):
        yield manipulate_latent(base_latent, eigenvector, magnitude), (magnitude)

# Generate interpolation
pbar = tqdm.tqdm(total=3*3*eigenvectors.shape[1])
for ei in range(eigenvectors.shape[1]):
    rows = []
    for idi in range(3):
        base_latent = torch.Tensor(test_pca[idi])
        interpolation = list(interpolate_eigenvector(base_latent, eigenvectors[:, ei]))
        images = []
        for i, (latent, magnitude) in enumerate(interpolation):
            # print (f"Interpolation {i+1}, magnitude: {magnitude}")
            image = generate_from_latent(latent)
            images.append(image)
            pbar.update(1)
        
        canvas = Image.new('RGB', (images[0].width*len(images), images[0].height))
        for i, image in enumerate(images):
            canvas.paste(image, (images[0].width*i, 0))
        
        rows.append(canvas)

    canvas = Image.new('RGB', (rows[0].width, rows[0].height*len(rows)))
    for i, row in enumerate(rows):
        canvas.paste(row, (0, row.height*i))
    imgbyte = io.BytesIO()
    canvas.save(imgbyte, format="JPEG", quality=20)
    canvas.save("/home/ubuntu/AutoLoRADiscovery/out/{}.jpg".format(ei), format="JPEG", quality=20)
    imgbyte.seek(0)


    display(IPImage(data=imgbyte.read(), width=canvas.width))
# interpolation = list(interpolate_eigenvector(base_latent, eigenvectors[:, 2]))
# for i, (latent, magnitude) in enumerate(interpolation):
#     print (f"Interpolation {i+1}, magnitude: {magnitude}")
#     generate_from_latent(latent)

In [ ]:
# # Generate images
base_latent = test_pca[0]
original_image = generate_from_latent(base_latent)
# manipulated_image = generate_from_latent(manipulated_latent)


# Interpolation along an eigenvector
def interpolate_eigenvector(base_latent, eigenvector, start=-2, end=2, steps=10):
    for magnitude in np.linspace(start, end, steps):
        yield manipulate_latent(base_latent, eigenvector, magnitude), (magnitude)

# Generate interpolation
interpolation = list(interpolate_eigenvector(base_latent, eigenvectors[:, 0]))
for i, (latent, magnitude) in enumerate(interpolation):
    print (f"Interpolation {i+1}, magnitude: {magnitude}")
    generate_from_latent(latent)

In [ ]:
blended_pca = torch.Tensor(latent_reconstructed[0]) #/ 2
decoded_lora = lora_vae.deapply_std_on_weights(lora_vae.decode(blended_pca.unsqueeze(0).cuda())).squeeze(0)
generate_from_weights(decoded_lora)

In [ ]:
blended_pca = (test_pca[0] + ((test_pca[1] * 9))) / 10
decoded_lora = lora_vae.deapply_std_on_weights(lora_vae.decode(blended_pca.unsqueeze(0).cuda())).squeeze(0)
generate_from_weights(decoded_lora)

In [ ]:
import torch
idens = torch.load("/mnt/rd/identity_df.pt", map_location="cpu")

In [ ]:
labels = []
for folder in list(idens.index): 
    labels.append(idens.loc[folder][label])

In [ ]:
idens.

In [ ]:
(idens.iloc[0].values[:-1]).shape

In [ ]:

# weights = torch.load("/mnt/rd/all_weights.pt", map_location="cuda")#.half()
V = torch.load("/mnt/rd/V.pt", map_location="cuda")#.half()
std = torch.load("/mnt/rd/std.pt", map_location="cuda")#.half()
std_w2w = torch.load("/mnt/rd/std_w2w.pt", map_location="cuda") * 5 #.half() 
mean = torch.load("/mnt/rd/mean.pt", map_location="cuda")#.half()
clip_embs = torch.load("/home/ubuntu/AutoLoRADiscovery/celeba_arc_embs.pt", map_location="cpu")


In [ ]:
latent_std = test_pca.std(dim=0)
latent_mean = test_pca.mean(dim=0)

In [ ]:
# save std and mean to ./latent_properties.pt
torch.save((latent_std,latent_mean), "./latent_properties.pt")

In [ ]:
normalized_latent = (test_pca - latent_mean) / latent_std

In [ ]:
normalized_latent.min(), normalized_latent.max(), normalized_latent.mean(), normalized_latent.std()

In [ ]:
test_pca.min(), test_pca.max(), test_pca.mean(), test_pca.std()

In [ ]:
import matplotlib.pyplot as plt
import numpy

# plt.plot((weights[0].unsqueeze(0)).squeeze(0).cpu().numpy())
# plt.show()
# plt.plot((apply_std_on_weights(weights[0].unsqueeze(0))).squeeze(0).cpu().numpy())
# # plt.show()
# plt.plot(((weights[0]).unsqueeze(0)).squeeze(0).cpu().numpy())
# plt.show()
# plt.plot(((reconstructed[0]).unsqueeze(0)).squeeze(0).cpu().numpy())
# plt.show()

# # # plt.plot(std_w2w.cpu().numpy())
# # # plt.show()

# # # plt.plot(std_w2w.abs().log().cpu().numpy()/10.0)
# # # plt.show()
# plt.plot(test_pca[1].cpu().numpy())
# plt.show()

# plt.plot(test_pca[0].cpu().numpy())
# plt.show()

plt.plot(test_pca[0].cpu().numpy())
plt.show()
plt.hist(test_pca[0:1000].flatten().cpu().numpy(), bins=100)
plt.show()
# plt.plot(normalized_latent[0].cpu().numpy())
# plt.show()
# # plt.hist(normalized_latent[0:10].cpu().numpy(), bins=100)
# # plt.show()
# plt.plot(latent_mean.cpu().numpy())
# plt.show()
# plt.plot(latent_std.cpu().numpy())
# plt.show()

In [ ]:

# clip_embs = torch.load("/home/ubuntu/AutoLoRADiscovery/celeba_arc_embs.pt", map_location="cpu")


In [ ]:
weight_dimensions

In [ ]:
segments = []
counter = 0
for key in weight_dimensions.keys():
    keyf = key.split(".")[2]+"."+key.split(".")[3]+"/"+key.split(".")[-4]+"/"+key.split(".")[-3]+"."+key.split(".")[-2]
    main_group=key.split(".")[2]+"."+key.split(".")[3]
    # if main_group not in segments:
    #     segments[main_group] = [[], 0]
    seg_std = weights[:, counter:counter+weight_dimensions[key][0][0]].std().item()
    segments.append((keyf, counter, counter+weight_dimensions[key][0][0], weight_dimensions[key][0][0], seg_std))
    # segments[main_group][1] += weight_dimensions[key][0][0]
    # final_weights0[key] = flattened_weights[0, counter:counter+weight_dimensions[key][0][0]].unflatten(0, weight_dimensions[key][1])
    counter += weight_dimensions[key][0][0]


In [ ]:
def apply_std_on_weights(weights):
    weight_parts = []
    for (key, start, end, length, std) in segments:
        weight_parts.append(weights[:, start:end] / std)
    return torch.cat(weight_parts, dim=-1)

def deapply_std_on_weights(weights):
    weight_parts = []
    for (key, start, end, length, std) in segments:
        weight_parts.append(weights[:, start:end] * std)
    return torch.cat(weight_parts, dim=-1)

In [ ]:
segments

In [ ]:
clip_embs["000005.jpg"].shape

In [ ]:
import tqdm
new_map = {}
new_weights = []
with torch.no_grad():
    for i, index in tqdm.tqdm(enumerate(idens.index.values), total=len(idens.index.values)):
        if index not in clip_embs:
            print ("skipping", index)
            continue
        ws = test_pca[i]#((weights[i] - mean) / std) @ V
        new_map[index] = (
            ws,
            clip_embs[index],#.half()
            idens.iloc[i].values,
            i
        )
        # new_weights.append(ws)
# new_weights = torch.stack(new_weights)
# save new map into /mnt/rd/celeba_map.pt
# torch.save(new_map, "/mnt/rd/celeba_map.pt")

# idens
# type(ids)
# len(ids.index.values)
# ids.iloc[:, 0]


In [ ]:
torch.save(new_map, "/mnt/rd/celeba_vae_map_clip_arc.pt")

In [ ]:
def w2w_to_lora(pca_coeffs):
    return ((pca_coeffs) @ V.T) * std + mean
recover_weight = w2w_to_lora(test_pca)

In [ ]:
torch.save(reconstructed,"/mnt/rd/reconstructed_weights.pt")

In [ ]:
test_pca.min(), test_pca.max(), test_weight.min(), test_weight.max(), recover_weight.min(), recover_weight.max()

In [ ]:
import os
from PIL import Image
device = "cuda:0"
if os.path.exists("/mnt/rd/inference_lora"):
    os.system("rm -rf /mnt/rd/inference_lora")

unflatten(((reconstructed[0])).unsqueeze(0).detach().clone(), weight_dimensions, "/mnt/rd/inference_lora")

pipe = DiffusionPipeline.from_pretrained("stablediffusionapi/realistic-vision-v51", 
                                        torch_dtype=torch.float16,safety_checker = None,
                                        requires_safety_checker = False).to(device)


pipe.unet = PeftModel.from_pretrained(pipe.unet, "/mnt/rd/inference_lora/unet", adapter_name="identity1")
adapters_weights1 = load_peft_weights("/mnt/rd/inference_lora/unet", device="cuda:0")
pipe.unet.load_state_dict(adapters_weights1, strict = False)
pipe.unet.to("cuda", torch.float16)



images = pipe(["A photo of a sks person"] * 4, height=640, width=640, num_inference_steps=50, guidance_scale=3.0).images
canvas = Image.new('RGB', (640*4, 640))
for i, image in enumerate(images):
    canvas.paste(image, (640*i, 0))

imgbyte = io.BytesIO()
canvas.save(imgbyte, format="JPEG", quality=20)
imgbyte.seek(0)
display(IPImage(data=imgbyte.read(), width=640*4))

In [ ]:
import tqdm
new_map = {}
new_weights = []
with torch.no_grad():
    for i, index in tqdm.tqdm(enumerate(idens.index.values), total=len(idens.index.values)):
        ws = ((weights[i] - mean) / std) @ V
        new_map[index] = (
            ws,
            clip_embs[index]
        )
        new_weights.append(ws)
new_weights = torch.stack(new_weights)
# save new map into /mnt/rd/celeba_map.pt
# torch.save(new_map, "/mnt/rd/celeba_map.pt")

# idens
# type(ids)
# len(ids.index.values)
# ids.iloc[:, 0]


In [ ]:
new_map[list(new_map.keys())[0]][0].shape

In [ ]:
new_weights.mean(), new_weights.std(), new_weights.shape, new_weights.min(), new_weights.max()

In [ ]:
mean = new_weights.mean()
centered_weights = new_weights - mean

# 2. Calculate the absolute maximum for symmetric scaling
max_abs_value = max(centered_weights.abs().max(), centered_weights.abs().min())

# 3. Scale the data to [-1, 1] range
scaled_weights = centered_weights / max_abs_value

In [ ]:
170_000

In [ ]:
print(scaled_weights.mean(), scaled_weights.std(), scaled_weights.shape, scaled_weights.min(), scaled_weights.max())


In [ ]:
max_value = max([torch.max(new_map[i][0]) for i in new_map])
min_value = min([torch.min(new_map[i][0]) for i in new_map])


In [ ]:
torch.save(new_map, "/mnt/rd/celeba_map.pt")

In [ ]:
new_map[list(new_map.keys())[10]][0].min(), new_map[list(new_map.keys())[10]][0].max()

In [ ]:
import torch

In [ ]:
ldd = torch.load("/mnt/rd/celeba_map.pt", map_location="cpu")

In [ ]:
tst_lora, key = ldd[list(ldd.keys())[0]][0], list(ldd.keys())[0]

In [ ]:
weights = torch.load("/mnt/rd/all_weights.pt", map_location="cpu")

In [ ]:
import sys
sys.path.append("/home/ubuntu/AutoLoRADiscovery/discover_lora_diffusion/weights2weights/")
from discover_lora_diffusion.weights2weights.lora_w2w import LoRAw2w
from discover_lora_diffusion.weights2weights.utils import unflatten
from diffusers import DiffusionPipeline 
from peft import PeftModel
from peft.utils.save_and_load import load_peft_weights

In [ ]:
weight_dimensions = torch.load("/mnt/rd/weight_dimensions.pt")

In [ ]:
unflatten(tst_lora.unsqueeze(0).clone().to("cuda"), weight_dimensions, "./example3")

In [ ]:
pipe = DiffusionPipeline.from_pretrained("stablediffusionapi/realistic-vision-v51", 
                                         torch_dtype=torch.float16,safety_checker = None,
                                         requires_safety_checker = False).to("cuda")
   
pipe.unet = PeftModel.from_pretrained(pipe.unet, "./example3/unet", adapter_name="identity1")
adapters_weights1 = load_peft_weights("./example3/unet", device="cuda:0")
pipe.unet.load_state_dict(adapters_weights1, strict = False)
pipe.to("cuda")
print("")

In [ ]:
from PIL import Image
#random seed generator
generator = torch.Generator(device="cuda")
generator = generator.manual_seed(6)
latents = torch.randn(
        (1, pipe.unet.in_channels, 512 // 8, 512 // 8),
        generator = generator,
        device = "cuda"
    ).half()

#inference parameters
prompt = "A photo of a sks person"
negative_prompt = "low quality, blurry, unfinished"
guidance_scale = 3.0
ddim_steps = 50
#run inference
image = pipe([prompt]*2, num_inference_steps=ddim_steps, guidance_scale=guidance_scale, height=640, width=640).images[0]
image

In [ ]:
key

In [ ]:
canvas.show()

In [ ]:
recon

In [ ]:
import diffusers

In [ ]:
config = {
    "_class_name": "UnCLIPScheduler",
    "_diffusers_version": "0.17.0.dev0",
    "clip_sample": True,
    "clip_sample_range": 4.0,  # Adjusted to match your data range
    "num_train_timesteps": 100,  # Reduced from 1000
    "prediction_type": "sample",
    "variance_type": "fixed_small_log"
}

scheduler = diffusers.UnCLIPScheduler.from_config(config)

In [ ]:
scheduler.betas

In [ ]:
import torch
import torchvision
import tqdm
import torchvision.transforms as transforms
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from PIL import Image
import warnings
image_transforms = transforms.Compose([transforms.Resize(512, interpolation=transforms.InterpolationMode.BILINEAR),
                                                transforms.RandomCrop(512),
                                                transforms.ToTensor(),
                                                transforms.Normalize([0.5], [0.5])])

In [ ]:
img = Image.open("/home/ubuntu/AutoLoRADiscovery/junk/1691527680903.jpeg")

In [ ]:
vae_input = image_transforms(img).unsqueeze(0).to("cuda").half()

In [ ]:
latents = pipe.vae.encode(vae_input).latent_dist.sample()

In [ ]:
latents_c = latents *0.18215

In [ ]:
latents_c.min(), latents_c.max(), latents_c.mean(), latents_c.std()

In [ ]:
latents.min(), latents.max(), latents.mean(), latents.std()

In [ ]:

import torch


model_id = "stabilityai/stable-diffusion-2-1"

# Use the DPMSolverMultistepScheduler (DPM-Solver++) scheduler here instead
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")


In [ ]:
pipe.scheduler.init_noise_sigma

In [ ]:
import diffusers
scheduler = diffusers.UnCLIPScheduler.from_config("kandinsky-community/kandinsky-2-2-prior", subfolder="scheduler")

In [ ]:
config = {
"_class_name": "UnCLIPScheduler",
"_diffusers_version": "0.17.0.dev0",
"clip_sample": True,
"clip_sample_range": 10.0,  # Adjusted to match your data range
"num_train_timesteps": 100,  # Reduced from 1000
"prediction_type": "epsilon",
"variance_type": "fixed_small_log"
}

custom_scheduler = diffusers.UnCLIPScheduler.from_config(config)

In [ ]:
scheduler.alphas

In [ ]:
scheduler.betas

In [ ]:
scheuler = diffusers.DDPMScheduler(num_train_timesteps=100, beta_start=0.0004, beta_end=0.999, beta_schedule="linear")

In [ ]:
scheuler.alphas

In [ ]:
scheuler.betas

In [ ]:
import numpy as np
from IPython.display import Image as IPImage, display
img_np = np.array(img)

def show_img_lq(img):
    img = Image.fromarray(img)
    import io
    buff_img = io.BytesIO()
    img = img.resize((img.width // 5, img.height // 5))
    img.save(buff_img, format="JPEG", quality=20) 
    buff_img.seek(0)
    display(IPImage(data=buff_img.read()))

In [ ]:
import tqdm
def show_noised_seq(img, scheduler):
    with torch.no_grad():
        img_list = []
        img_tensor = image_transforms(img).unsqueeze(0).to("cuda") 
        std_img = img_tensor.std()
        mean_img = img_tensor.mean()
        
        img_tensor = (img_tensor - mean_img) / std_img
        
        for t in tqdm.trange(scheduler.num_train_timesteps):
            noise = torch.randn_like(img_tensor) 
            img_noisy = scheduler.add_noise(img_tensor, noise, torch.Tensor([t]).long())
            img_list.append((img_noisy, noise, t))
        _img_list = [img_list[0]]
        stp_v =(scheduler.num_train_timesteps//30)
        for i in range(0, scheduler.num_train_timesteps, stp_v):
            _img_list.append(img_list[i: i+stp_v][-1])
        # _img_list.append(img_list[-1])

        def show_imglistt(_img_list):
            merged = torch.cat(_img_list, dim=3)
            lv = _img_list[-1][0]
            fv = _img_list[0][0]
            print (lv.min(), lv.max(), lv.mean(), lv.std())
            print (fv.min(), fv.max(), fv.mean(), fv.std())
            print(merged.shape, merged.min(), merged.max(), merged.mean(), merged.std())
            merged = (merged * std_img) + mean_img
            merged = merged.cpu().numpy().squeeze() 
            merged = (merged + 1) / 2
            merged = (merged * 255).astype(np.uint8)
            merged = np.transpose(merged, (1,2,0))
            show_img_lq(merged)
        
        show_imglistt([img_item[0] for img_item in _img_list])

        # reversed
        _reverse_img_list = []
        for i in range(len(_img_list)-1, -1, -1):
            (noised, noise, t) = _img_list[i]
            print(t)
            # noised = scheduler.scale_model_input(noised, t)
            denoised = scheduler.step(noise, t, noised, return_dict=False)[0]
            _reverse_img_list.append(denoised)
        show_imglistt(_reverse_img_list)

In [ ]:
# pipe.scheduler.set_timesteps(100)
show_noised_seq(img, custom_scheduler)

In [ ]:
for i in range(9, -1, -1):
    print(i)

In [ ]:
xl = torch.randn(1, 3, 512, 512)
print(xl.shape, xl.min(), xl.max(), xl.mean(), xl.std())

In [ ]:
scheduler.timesteps